In [0]:
-- This reads the raw IoT device data as it arrives
CREATE OR REFRESH STREAMING LIVE TABLE fitness_devices_bronze
COMMENT "Raw IoT fitness device data from JSON files"
AS SELECT 
  *,
  current_timestamp() as ingestion_time,
  _metadata.file_path as source_file
FROM cloud_files(
  "/databricks-datasets/iot-stream/data-device/*.json",
  "json",
  map("cloudFiles.inferColumnTypes", "true")
);

In [0]:
-- This creates a cleaned, validated version of the data
CREATE OR REFRESH STREAMING LIVE TABLE fitness_devices_silver
(
  -- Data quality rules: drop bad records
  CONSTRAINT valid_steps EXPECT (num_steps >= 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_calories EXPECT (calories_burnt >= 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_miles EXPECT (miles_walked >= 0) ON VIOLATION DROP ROW,
  CONSTRAINT reasonable_steps EXPECT (num_steps < 100000) ON VIOLATION DROP ROW
)
COMMENT "Cleaned fitness data with quality checks"
AS SELECT
  device_id,
  user_id,
  num_steps,
  miles_walked,
  calories_burnt,
  -- Convert timestamp string to proper timestamp
  to_timestamp(timestamp) as activity_timestamp,
  -- Extract useful time parts
  date(to_timestamp(timestamp)) as activity_date,
  hour(to_timestamp(timestamp)) as activity_hour,
  -- Calculate steps per mile (if miles > 0)
  CASE 
    WHEN miles_walked > 0 THEN num_steps / miles_walked 
    ELSE 0 
  END as steps_per_mile,
  -- Categorize activity level
  CASE
    WHEN num_steps < 1000 THEN 'Low'
    WHEN num_steps BETWEEN 1000 AND 5000 THEN 'Moderate'
    WHEN num_steps BETWEEN 5001 AND 10000 THEN 'Active'
    ELSE 'Very Active'
  END as activity_level
FROM STREAM(LIVE.fitness_devices_bronze)
WHERE device_id IS NOT NULL 
  AND user_id IS NOT NULL;

In [0]:
-- Daily user summary
CREATE OR REFRESH LIVE TABLE daily_user_summary
COMMENT "Daily activity summary per user"
AS SELECT
  user_id,
  activity_date,
  COUNT(DISTINCT device_id) as devices_used,
  SUM(num_steps) as total_steps,
  SUM(miles_walked) as total_miles,
  SUM(calories_burnt) as total_calories,
  AVG(num_steps) as avg_steps_per_reading,
  MAX(num_steps) as max_steps_in_reading,
  -- Daily goal achievement (10,000 steps)
  CASE 
    WHEN SUM(num_steps) >= 10000 THEN 'Goal Achieved'
    ELSE 'Below Goal'
  END as daily_goal_status
FROM LIVE.fitness_devices_silver
GROUP BY user_id, activity_date;

-- Hourly activity patterns
CREATE OR REFRESH LIVE TABLE hourly_activity_patterns
COMMENT "Activity patterns throughout the day"
AS SELECT
  activity_hour,
  COUNT(DISTINCT user_id) as active_users,
  AVG(num_steps) as avg_steps,
  AVG(calories_burnt) as avg_calories,
  SUM(num_steps) as total_steps,
  -- Identify peak hours
  CASE
    WHEN activity_hour BETWEEN 6 AND 9 THEN 'Morning'
    WHEN activity_hour BETWEEN 12 AND 13 THEN 'Lunch'
    WHEN activity_hour BETWEEN 17 AND 19 THEN 'Evening'
    WHEN activity_hour BETWEEN 20 AND 22 THEN 'Night'
    ELSE 'Other'
  END as time_period
FROM LIVE.fitness_devices_silver
GROUP BY activity_hour
ORDER BY activity_hour;

-- User fitness leaderboard
CREATE OR REFRESH LIVE TABLE user_fitness_rankings
COMMENT "Overall user fitness rankings"
AS SELECT
  user_id,
  COUNT(DISTINCT activity_date) as active_days,
  COUNT(DISTINCT device_id) as devices_used,
  SUM(num_steps) as lifetime_steps,
  SUM(miles_walked) as lifetime_miles,
  SUM(calories_burnt) as lifetime_calories,
  AVG(num_steps) as avg_daily_steps,
  -- Rank users by total steps
  RANK() OVER (ORDER BY SUM(num_steps) DESC) as steps_rank,
  -- Classify user fitness level
  CASE
    WHEN AVG(num_steps) >= 10000 THEN 'Highly Active'
    WHEN AVG(num_steps) >= 7500 THEN 'Active'
    WHEN AVG(num_steps) >= 5000 THEN 'Moderately Active'
    ELSE 'Low Activity'
  END as fitness_category
FROM LIVE.fitness_devices_silver
GROUP BY user_id;

-- Device reliability analysis
CREATE OR REFRESH LIVE TABLE device_performance
COMMENT "Analysis of device reporting patterns"
AS SELECT
  device_id,
  user_id,
  COUNT(*) as total_readings,
  COUNT(DISTINCT activity_date) as days_active,
  MIN(activity_timestamp) as first_seen,
  MAX(activity_timestamp) as last_seen,
  -- Calculate average readings per day
  COUNT(*) / NULLIF(COUNT(DISTINCT activity_date), 0) as avg_readings_per_day
FROM LIVE.fitness_devices_silver
GROUP BY device_id, user_id;